# Task 3: Diverse Explanations

## Loading Data & Preparation

In [1]:
# Load bug reports explanations

import os
from pathlib import Path
import pandas as pd
import json

data = pd.read_csv("../../data/answerList_data.csv")

bug_reports_data = {}

# read one file per failing method
for failing_method in data['FailingMethod'].unique():
    with open(Path(f"../exercise2/{failing_method}.txt"), 'r') as f:
        # individual explanations are separated by a newline
        explanations = f.read().split("\n")
        explanations = [explanation for explanation in explanations if explanation != ""]
        bug_reports_data[failing_method] = explanations   

print(bug_reports_data["HIT01_8"])  

['Minutes are set to -15; which is less then 0 and it throws illegal arg exception', 'The code never gets that far. The problem is at line 279 which prevents a negative minutes value being accepted even though the programmer comments indicate that since version 2.3 negative minutes up to -59 are acceptable. The @throws IllegalArgumentException comment is also referring to versions before 2.3.', 'In the code there is a check that 0 <= minutes < 60 and the minutesOffset is -15 which does not fall into these prarmeters thus throwing an Exception', 'There is a logical check for if minuteOffset is less than 0 or greater than 59 causing it to throw an exception because the value is out of bounds (negative number)', 'YES. The issue is on line 279 (as I explained in my first question; of which I misunderstood that I was only being asked about the specific issue; not generalized issue). On line 279 the variable "minutesOffSet" is parameterized to throw an exception if it is < 0 or > 59. Line 27

In [2]:
# load the ground truth explanations
with open("ground_truth_explanations.json", "r") as f:
    ground_truth_explanations = json.load(f)

In [3]:
# calculate readability scores for the ground truth explanations

import textstat
readability_scores = {}
for method, explanation in ground_truth_explanations.items():
    readability_scores[method] = textstat.flesch_reading_ease(explanation)

# sort the explanations by readability
sorted_explanations = sorted(readability_scores.items(), key=lambda x: x[1], reverse=True)

# print readibility scores
for method, score in sorted_explanations:
    print(f"{method}: {score}")

readability_threshold = {}
for method, score in readability_scores.items():
    readability_threshold[method] = round(score * 0.8,3)

# print readability thresholds
for method, score in readability_threshold.items():
    print(f"{method}: {score}")

HIT03_6: 64.75
HIT05_35: 58.62
HIT07_33: 53.04
HIT02_24: 51.18
HIT04_7: 50.16
HIT08_54: 46.81
HIT06_51: 43.02
HIT01_8: 28.67
HIT01_8: 22.936
HIT02_24: 40.944
HIT03_6: 51.8
HIT04_7: 40.128
HIT05_35: 46.896
HIT06_51: 34.416
HIT07_33: 42.432
HIT08_54: 37.448


In [4]:
# define simialrity
from sentence_transformers import SentenceTransformer, util
import numpy as np

# Load a high-performance model for semantic similarity
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

def compute_similarity(ground_truth, user_explanation):
    """
    Computes cosine similarity between the ground truth explanation and a user-given explanation.
    """
    embeddings = model.encode([ground_truth, user_explanation], convert_to_tensor=True)
    similarity = util.pytorch_cos_sim(embeddings[0], embeddings[1])
    return similarity.item()

c:\Users\bzwad\anaconda3\envs\DSWebscrape\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# Compute similarity for all user explanations and set threshold
similarity_scores = {}

for method, ground_truth in ground_truth_explanations.items():
    scores = [compute_similarity(ground_truth, user_exp) for user_exp in bug_reports_data.get(method, [])]
    similarity_scores[method] = scores

# average similarity per failing method
average_similarity_scores = {}
for method, scores in similarity_scores.items():
    average_similarity_scores[method] = sum(scores) / len(scores)

for method, score in average_similarity_scores.items():
    print(f"{method}: {score}")

# set similarity threshold to 70th percentile
similarity_threshold = {}
for method, scores in similarity_scores.items():
    similarity_threshold[method] = np.percentile(scores, 70)

for method, score in similarity_threshold.items():
    print(f"{method}: {score}")

HIT01_8: 0.5436896428233012
HIT02_24: 0.3701791672501713
HIT03_6: 0.25586818529409355
HIT04_7: 0.2914702493418008
HIT05_35: 0.42632644800469277
HIT06_51: 0.30135327534129225
HIT07_33: 0.4918408831271032
HIT08_54: 0.3938455417131384
HIT01_8: 0.7063461601734161
HIT02_24: 0.5649046778678893
HIT03_6: 0.32944928407669066
HIT04_7: 0.38390407562255857
HIT05_35: 0.5162835419178009
HIT06_51: 0.4160767793655395
HIT07_33: 0.6244548857212067
HIT08_54: 0.4770480155944824


In [6]:
# define utility functions

import re
from pathlib import Path
import enchant

def contains_english_word(text):
    """
    Check if a string contains at least one English word.
    Returns True if an English word is found, False otherwise.
    
    Args:
        text (str): Input string to check
        
    Returns:
        bool: True if contains English word, False otherwise
    """
    # Initialize English dictionary
    dictionary = enchant.Dict("en_US")
    
    # Clean the string - keep only letters and spaces
    cleaned_text = re.sub(r'[^a-zA-Z\s]', ' ', text)
    
    # Split into potential words
    words = cleaned_text.split()
    
    # Check each potential word
    return any(dictionary.check(word.lower()) for word in words)

## Diversity of Explanations

(Q 3.1) There are many different ways which we could use to measure the diversity of explanations. We could, for example, use the type-token ratio or the number of unique words in the explanations to measure the diversity of words used in the explanations. Since we did not get great results with the TTR in previous experiments, we will not use this approach. Instead, we want to regard the Shannon Entropy of the explanations. The Shannon Entropy is a measure of the uncertainty in a random variable. In our case, the random variable is the choice of words in the explanations. The Shannon Entropy is defined as follows:

$$H(X) = - \sum_{i=1}^{n} p(x_i) \cdot \log_2(p(x_i))$$

where $p(x_i)$ is the probability of the $i$-th word in the explanation. Shannon entropy measures the randomness or uncertainty in a distribution—in this case, the distribution of words in a text. Higher entropy means more unpredictability, while lower entropy suggests redundancy or repetition. The minimum entropy is 0, which occurs when all words are the same. The maximum entropy is the logarithm of the number of words in the text, which occurs when all words are equally likely. For normal written English, entropy usually ranges between 4 and 8 bits per word, depending on vocabulary richness.

We could also measure diversity by looking at the semantic diversity of explanations using the embedding distance (mean pairwise cosine similarity) between the explanations. Since this directly contradicts a high similarity between explanations, we decided against this approach.
In the broader range of diversity, we could look at more features than the explanation itself, e.g. at associated categorical features like the bug reporters demographic data, their experience level or the country that they are from. However, this makes it harder to define exact thresholds for diversity.

As the Shannon Entropy seems like the most straight forward approach, we will use this to measure the diversity of explanations in the following.

In [24]:
from collections import Counter
import math

def shannon_entropy(text):
    words = text.split()
    word_counts = Counter(words)
    total_words = len(words)
    entropy = -sum((count/total_words) * math.log2(count/total_words) for count in word_counts.values())
    return entropy

In [25]:
# calculate shannon entropy for ground truth explanations
ground_truth_entropy = {}
for method, explanation in ground_truth_explanations.items():
    ground_truth_entropy[method] = shannon_entropy(explanation)

# sort the explanations by entropy
sorted_entropy = sorted(ground_truth_entropy.items(), key=lambda x: x[1], reverse=True)

# print entropy scores
for method, score in sorted_entropy:
    print(f"{method}: {score}")

HIT08_54: 5.358714497742255
HIT07_33: 5.25585347326784
HIT06_51: 5.115114023681427
HIT01_8: 5.10341455748809
HIT03_6: 4.999664476749764
HIT05_35: 4.898153434632013
HIT02_24: 4.8219280948873635
HIT04_7: 4.812209613812088


We can see that all our ground truth explanations achieve a Shannon Entropy of 4-6 bits per word. We will use this as a reference point for the diversity of explanations. This moderate Entropy is in general desirable, as it indicates a good balance between clarity and lexical variety.

(Q 3.2) The max readability and max similarity values independent of the diversity per bug report are as follows:

In [26]:
# Process all methods in a single pass to find highest similarity and readability scores/explanations
highest_similarity_scores = {}
highest_readability_scores = {}
most_similar_explanations = {}
most_readable_explanations = {}

for method, explanations in bug_reports_data.items():
    # Calculate similarity scores for all explanations
    similarity_scores = [compute_similarity(ground_truth_explanations[method], explanation) for explanation in explanations]
    max_similarity_index = similarity_scores.index(max(similarity_scores))
    
    # Store highest similarity score and corresponding explanation
    highest_similarity_scores[method] = similarity_scores[max_similarity_index]
    most_similar_explanations[method] = explanations[max_similarity_index]
    
    # Calculate readability scores for all explanations
    readability_scores = []
    for explanation in explanations:
        if len(explanation.split()) == 0 or not contains_english_word(explanation) or len(explanation.split())<4:
            readability_scores.append(0)
        else:
            readability_scores.append(textstat.flesch_reading_ease(explanation))
    
    max_readability_index = readability_scores.index(max(readability_scores))
    
    # Store highest readability score and corresponding explanation
    highest_readability_scores[method] = readability_scores[max_readability_index]
    most_readable_explanations[method] = explanations[max_readability_index]

# Print scores
for method, (similarity, readability) in zip(highest_similarity_scores.keys(), 
                                           zip(highest_similarity_scores.values(), 
                                               highest_readability_scores.values())):
    print(f"{method}: {similarity}, {readability}")

HIT01_8: 0.820572555065155, 99.23
HIT02_24: 0.7392517328262329, 100.24
HIT03_6: 0.6439655423164368, 118.18
HIT04_7: 0.7673725485801697, 118.18
HIT05_35: 0.7223565578460693, 104.64
HIT06_51: 0.6698489785194397, 116.15
HIT07_33: 0.7574557662010193, 118.18
HIT08_54: 0.7468466758728027, 110.06


The readability scores are quite high because some people put very simple "explanations" like "yes" or "no" as their answer. To avoid artificially inflated scores, we filtered out explanations that did not contain English words or were shorter than 4 words.
However, the most readable explanations still tend to be very simple answers, indicating that readability alone might not be the best metric to optimize for.

In [27]:
print(most_readable_explanations["HIT07_33"])
print(most_readable_explanations["HIT04_7"])
print(most_readable_explanations["HIT05_35"])

i++ should be i+
I am not sure.
type will be object when it should be string


(Q 3.3) To find the max diversity for the max similarity (compromising readability) we first look directly at the diversity of the methods with the highest similarity:

In [28]:
entropies = {}

for method, explanation in most_similar_explanations.items():
    entropies[method] = shannon_entropy(explanation)
    print(f"{method}: {entropies[method]}")

HIT01_8: 4.829966150010236
HIT02_24: 4.720049960644813
HIT03_6: 6.022673958060609
HIT04_7: 5.519999987133631
HIT05_35: 6.443530320573129
HIT06_51: 5.576098444107486
HIT07_33: 4.004886164091841
HIT08_54: 5.490861597695111


In [29]:
# comparing highest similarity entropies to ground_truth_entropy 

for method, entropy in entropies.items():
    print(f"{method}: {entropy} vs {ground_truth_entropy[method]} (GT), deviation = {ground_truth_entropy[method]- entropy}")

HIT01_8: 4.829966150010236 vs 5.10341455748809 (GT), deviation = 0.27344840747785426
HIT02_24: 4.720049960644813 vs 4.8219280948873635 (GT), deviation = 0.10187813424255054
HIT03_6: 6.022673958060609 vs 4.999664476749764 (GT), deviation = -1.0230094813108446
HIT04_7: 5.519999987133631 vs 4.812209613812088 (GT), deviation = -0.7077903733215436
HIT05_35: 6.443530320573129 vs 4.898153434632013 (GT), deviation = -1.545376885941116
HIT06_51: 5.576098444107486 vs 5.115114023681427 (GT), deviation = -0.46098442042605914
HIT07_33: 4.004886164091841 vs 5.25585347326784 (GT), deviation = 1.2509673091759987
HIT08_54: 5.490861597695111 vs 5.358714497742255 (GT), deviation = -0.13214709995285556


As we can see, for all of these cases the diversity is acceptable, although it is mostly slightly lower than our ground truth. Because of this, we decided to also look at the top 10 most similar explanations for each bug report and calculate the Shannon Entropy for each of them. We also regard combinations of these explanations to find the highest diversity for the highest similarities, i.e. the highest diversity may be achieve by combining the top 5 most similar explanations or by only taking one of the top 10 most similar explanations.

In [30]:
from itertools import combinations

top10_similar_explanations = {}
for method, explanations in bug_reports_data.items():
    similarity_scores = [compute_similarity(ground_truth_explanations[method], explanation) for explanation in explanations]
    top10_indices = sorted(range(len(similarity_scores)), key=lambda i: similarity_scores[i], reverse=True)[:10]
    top10_similar_explanations[method] = [explanations[i] for i in top10_indices]

# find the combination with the highest shannon entropy per bug report
highest_entropy = {}
highest_entropy_explanations = {}

for method, explanations in top10_similar_explanations.items():
    max_entropy = 0
    max_entropy_explanations = []
    
    # Check combinations of different lengths
    for length in range(1, len(explanations) + 1):
        for combination in combinations(explanations, length):
            combined_text = " ".join(combination)
            entropy = shannon_entropy(combined_text)
            if entropy > max_entropy:
                max_entropy = entropy
                max_entropy_explanations = combination
    
    highest_entropy[method] = max_entropy
    highest_entropy_explanations[method] = max_entropy_explanations

# print highest entropy scores
for method, (score, explanations) in zip(highest_entropy.keys(), 
                                           zip(highest_entropy.values(), 
                                               highest_entropy_explanations.values())):
    print(f"{method}: {score}, explanations ({len(explanations)}): {explanations}")

HIT01_8: 7.104215726535024, explanations (10): ('The variable "minutesOffset" is checked incorrectly by the IF statement on line 279. Any negative value for "minutesOffset" will throw this exception; while the documentation states that "minutesOffset" can be negative in some cases.', 'There is a logical check for if minuteOffset is less than 0 or greater than 59 causing it to throw an exception because the value is out of bounds (negative number)', 'The code never gets that far. The problem is at line 279 which prevents a negative minutes value being accepted even though the programmer comments indicate that since version 2.3 negative minutes up to -59 are acceptable. The @throws IllegalArgumentException comment is also referring to versions before 2.3.', 'In the code there is a check that 0 <= minutes < 60 and the minutesOffset is -15 which does not fall into these prarmeters thus throwing an Exception', 'The minute variable was negative therefore; it threw the exception because it on

In [31]:
# determine average readability scores for the highest entropy explanations
highest_entropy_readability = {}
for method, explanations in highest_entropy_explanations.items():
    readability_scores = [textstat.flesch_reading_ease(explanation) for explanation in explanations]
    highest_entropy_readability[method] = sum(readability_scores) / len(readability_scores)

# print highest entropy readability scores
for method, score in highest_entropy_readability.items():
    print(f"{method}: {score}")

HIT01_8: 59.39300000000001
HIT02_24: 67.53
HIT03_6: 65.34555555555555
HIT04_7: 49.75888888888889
HIT05_35: 70.2457142857143
HIT06_51: 78.22
HIT07_33: 62.59375
HIT08_54: 56.3


This approach seems to favor multiple and thus slightly longer explanations. With this, we reach rather high entropy scores. This is a good sign, as it indicates that the explanations are diverse and not just repetitions of the same information. The average readability is a bit low, however, this is a trade-off that we are willing to make to ensure that the explanations are diverse. We would also expect this value to increase when feeding the explanations to the LLM.

### Conclusion
Combining these insights with our previous results for task 2, it seems logical to curate a set of ~10 explanations for each bug report, which are the most similar to the ground truth and have a high diversity. This way, we can ensure that the explanations are both accurate and diverse. Since readability already comes to some extend with the similarity, it can be neglected or only slightly optimized for. We can also imagine that the readability can easily be tweaked by prompting the LLM to generate more readable explanations, while similar approaches to similarity and diversity are not as straight forward.